In [1]:
from bs4 import BeautifulSoup as bs
from itertools import cycle
import random
import collections
from random import choice
import time
import requests
import pandas as pd
import re

In [5]:
proxy_url = 'https://github.com/saschazesiger/Free-Proxies/blob/master/proxies/ultrafast.txt'

In [6]:
def get_proxies(url):

    response = requests.get(url)
    soup = bs(response.content, 'lxml').find_all('td',{'class':'blob-code blob-code-inner js-file-line'})
    proxies = ["http://"+i.text for i in soup]

    keys = []
    for i in range(len(proxies)):
        keys.append('http')

    proxy_list = [{L2 : L1} for (L1,L2) in zip(proxies, keys)]  
    
    return proxy_list

In [7]:
proxy_list = get_proxies(proxy_url)

In [13]:
del proxy_index

In [9]:
proxy_index = 0

In [10]:
def proxy_check(url, proxy_list):
    global proxy_index
    '''
    #working = []
    for i in range(0,100):
    while True:
        try:
            # do stuff
        except SomeSpecificException:
            continue
        break
    #for i in range(100):
    '''
    working = []
    while proxy_index < len(proxy_list)/2000:
    #for i in proxy_list[:25]:
        #proxy = get_random_proxy(new_proxies)
        #x = choice(proxy_list)
        try:
            #print(f"Using {x} ...")
            print(f"Using {proxy_list[proxy_index]} ...")
            #time.sleep(random.randint(10,30))
            r = requests.get(url ,proxies=proxy_list[proxy_index], timeout=3)
            #r = requests.get(url ,proxies=x, timeout=3)
            #working.append(proxy_list[proxy_index])
            print(r.status_code)
        #print(r)
            return r
        except:
            proxy_index += 1
            print('Bad proxy..')
            print(proxy_index)
            
      

In [12]:

tech_url = 'https://www.techpowerup.com'
test_url = 'https://www.google.com/'

#source = proxy_check(tech_url, proxy_list)
source = proxy_check(test_url, proxy_list)

    

In [ ]:
print(source)

In [ ]:
del links

In [ ]:
tech_url = 'https://www.google.com/'
# 'https://www.techpowerup.com/gpu-specs/'
#proxy = choice(proxy_check()) /gpu-specs/

source = proxy_check(tech_url, proxy_list)

In [ ]:
print(source)

In [ ]:
tech_url = 'https://www.techpowerup.com/gpu-specs/'
#proxy = choice(proxy_check()) /gpu-specs/
source = requests.get(tech_url , timeout=10)
#source = proxy_check(tech_url, proxy_list)
#proxy_index += 1
print(source.status_code)
src = source.content
soup = bs(src,'lxml')

links = []
for i in soup.find('div', class_="table-wrapper").find_all('tr'):
    next_page = i.find('a')
    if next_page != None:
        #time.sleep(random.randint(0,3)).strip('/gpu-specs/').strip('/gpu-spec/')
        page_list = next_page.attrs['href']
        print(page_list)
        links.append(tech_url.strip('/gpu-spec/')+page_list)
     
    
    #


In [ ]:
print(links)

In [ ]:
counter = 0
title_list = []
col_label_list = []
data_list = []
while True:
    for i in links:
        if counter <= len(links):
            time.sleep(random.randint(3,5))
            source = proxy_check(tech_url, proxy_list)
            #source = requests.get(i, proxies = proxy)
            print(source.status_code)
            #if source.status_code == 429:
                #proxy_index += 1
            src = source.content
            soup = bs(src,'lxml')
            print(f"-----------{counter}----------")
            #title = soup.find('h1', class_="gpudb-name").text
            #title_list.append(title)
            #print(title)
            stats = soup.find('div').find_all('dl')
            for j in stats:
                column_label = j.find('dt').text
                col_label_list.append(column_label)
                data = j.find('dd').text.replace('\n',"").replace('\t',"")
                data_list.append(data)
            counter+=1
    break

In [ ]:
print(title_list)
print(col_label_list)
print(data_list)

In [ ]:
title = soup2.find('h1', class_="gpudb-name").text
#specs = soup2.find('dd', class_="gpudb-specs-large__value")
specs1 = soup2.find('div').find_all('dl')
#specs2 = soup2.find('dl', class_="clearfix").find_all('div')
#specs1 = soup2.find('dl', class_="gpudb-specs-large").find_all('div')
print(title)
list_info = []
list_info_heading = []
for spec1 in specs1:
    basic_info = spec1.find('dd').text.replace('\n',"").replace('\t',"")
    basic_heading = spec1.find('dt').text
    list_info.append(basic_info)
    list_info_heading.append(basic_heading)
    
print(list_info_heading)
#print(list_info)

#for i in list_info:
    #print(list_info[i].strip(','))

keep = []
pattern_sub = ',|,'
pattern_find = '\d+\.*\d*|$'
#pattern_find = '\d+|$'

test_list = [re.sub(pattern_sub, '', i) for i in list_info]
test_list2 = [re.findall(pattern_find, i)[0] for i in test_list]
# re.findall('\d+|$', 'aa33bbb44')[0]
#for i in list_info:'/^[+-]?([0-9]+\.?[0-9]*|\.[0-9]+)$/'
#x = [re.findall(pattern, ' ', i) for i in list_info]
#x = list([re.findall(pattern, list_info) for i in list_info])
#pattern = '[0-9]'
#test_list = [re.sub(pattern, '', i) for i in list_info]

#string = ''.join(list_info)
#match = re.findall(pattern, string)
# Pattern.compile("(^|\\s)([0-9]+)($|\\s)");
# match = re.findall(r"\([A-Z]+\)|Pool ID", string)

  

#print(x)    
    
#print(x)
#print(keep)

    
#print(title)

    
    #test = spec.find('dd', class_='gpudb-specs-large__value').text \n\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t
    #print(list_value) ,class_='gpudb-specs-large__value'
    # ,class_='gpudb-specs-large__title' replace("geeks", "GeeksforGeeks", 3))

#print(specs1)   .strip('\t').strip('\n').strip()
    
#movies = soup.find('tbody', class_="lister-list").find_all('tr')

In [ ]:
print(test_list)  

In [ ]:
print(test_list2)

In [ ]:
#df = pd.read_csv(names=io.StringIO('\n'.join(list_info_heading)), delim_whitespace=False)
#df = pd.DataFrame(list_info_heading)
#pd.DataFrame.from_dict(data)
temp_dict = {}
for key, value in zip(list_info_heading, test_list2):
    temp_dict[key] = value  
        
df = pd.DataFrame(temp_dict, index=[0])


#print(list(df.columns))
drop_list = ['GPU Name', 'GPU Variant', 'Architecture', 'Foundry', 'Production', 'Slot Width', 
             'Length', 'Width', 'Height', 'Outputs', 'DirectX', 'OpenGL', 'OpenCL', 'Vulkan', 
             'Shader Model','Power Connectors','Part Number','Suggested PSU','Predecessor',
             'L0 Cache', 'L1 Cache', 'L2 Cache', 'L3 Cache']
#columns =['C', 'D']
df['Graphics Processor'] = title
df = df.drop(columns = drop_list)
#df = df.convert_dtypes()
#df = df.convert_dtypes().dtypes
#df = pd.DataFrame(temp_dict)
#df = df.append(temp_dict, ignore_index=True, sort=False)

In [ ]:
df.info()

In [ ]:
try:
    source = requests.get('https://www.imdb.com/chart/top/')
    source.raise_for_status()
    soup = BeautifulSoup(source.text,'html.parser')
    movies = soup.find('tbody', class_="lister-list").find_all('tr')
    print(movies)
    df = pd.DataFrame(columns=['Rank','Name','Year','Rating'])
    temp_dict = {}
    temp_list = []
    
    for movie in movies:
        name = movie.find('td', class_='titleColumn').a.text
        rank = movie.find('td', class_='titleColumn').get_text(strip=True).split('.')[0]
        year = movie.find('td', class_='titleColumn').span.text.strip('()')
        rating = movie.find('td', class_='ratingColumn imdbRating').strong.text
        print(name)
        list_key = ['Rank','Name','Year','Rating']
        list_value = [rank, name, year, rating]
        
        for key, value in zip(list_key, list_value):
            temp_dict[key] = value
            
        df = df.append(temp_dict, ignore_index=True, sort=False)
   
except Exception as e:
    print(e)
    


In [ ]:
df.head()